# ECM-MPT Data Analysis

Extracellular-Matrix Multiple Particle Tracking data analysis and prediciton of pup age in P14, P21, P28, and P35 datasets. 


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.---Load-Data" data-toc-modified-id="1.---Load-Data-1">1.   Load Data</a></span></li><li><span><a href="#2.---Analysis" data-toc-modified-id="2.---Analysis-2">2.   Analysis</a></span></li><li><span><a href="#3.---Modelling" data-toc-modified-id="3.---Modelling-3">3.   Modelling</a></span></li><li><span><a href="#4.---Evaluate-Results" data-toc-modified-id="4.---Evaluate-Results-4">4.   Evaluate Results</a></span></li></ul></div>

---

## 1.   Load Data

In [4]:
# pd.read ...

## 2.   Analysis

In [5]:
# df = ...

## 3.   Modelling

In [3]:
# cld ...

## 4.   Evaluate Results